<a href="https://colab.research.google.com/github/lslbastos/Brazil_Vaccine_PNI/blob/main/Backup_Brazil_Vaccine_DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
## Importing functions
from src.get_page_links import get_page_links
from src.get_pni_data_import import get_pni_data_csv

get_page_links()

In [ ]:
## Defining target state
state = 'PA'

## Obtaining State data
df_si_pni = get_pni_data_csv(get_page_links()[state])




In [ ]:
df_si_pni

,document_id,paciente_id,paciente_idade,paciente_datanascimento,paciente_enumsexobiologico,paciente_racacor_codigo,paciente_racacor_valor,paciente_endereco_coibgemunicipio,paciente_endereco_copais,paciente_endereco_nmmunicipio,paciente_endereco_nmpais,paciente_endereco_uf,paciente_endereco_cep,paciente_nacionalidade_enumnacionalidade,estabelecimento_valor,estabelecimento_razaosocial,estalecimento_nofantasia,estabelecimento_municipio_codigo,estabelecimento_municipio_nome,estabelecimento_uf,vacina_grupoatendimento_codigo,vacina_grupoatendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataaplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds,id_sistema_origem
0,06742d02-09be-43c7-8748-757eb07e0a78-i0b0,399cc354805faae31f081f8e3ff8310c3ad98529ba9e78...,59.0,1962-04-17,F,3.0,PARDA,150680.0,10.0,SANTAREM,BRASIL,PA,68025.0,B,3612392,PREFEITURA MUNICIPAL DE SANTAREM,UNIDADE DE VIGILANCIA EPIDEMIOLOGICA,150680,SANTAREM,PA,106,Doenças Cardiovasculares e Cerebrovasculares,1.0,Comorbidades,ABW4735,MINISTERIO DA SAUDE,Organization/00394544000851,2021-05-11,1ª Dose,89,Covid-19-AstraZeneca,Novo PNI,2021-05-17T16:53:33.000Z,16341
1,15f6f2b8-a79f-40ea-86ba-76be300a9b01-i0b0,64de84a36ce5c6e57166c5d39b2361d4c4f180ce785981...,21.0,2000-01-14,F,4.0,AMARELA,150613.0,10.0,REDENCAO,BRASIL,PA,68551.0,B,7699115,MUNICIPIO DE REDENCAO,ANTONIO JOSE SOARES,150613,REDENCAO,PA,926,Outros,9.0,Trabalhadores de Saúde,215VCD144W,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-07-14,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-08-19T12:38:00.000Z,16341
2,5a24959a-d02f-4aa1-b627-2fc836c78789-i0b0,bae0655b014792e11eb9459a493df34d095abe6ab2cc47...,48.0,1973-05-12,F,4.0,AMARELA,150320.0,10.0,IGARAPE-ACU,BRASIL,PA,68725.0,B,2312255,PREFEITURA MUNICIPAL DE IGARAPEACU,UNIDADE BASICA DE SAUDE VILA CURY,150320,IGARAPE-ACU,PA,107,Hipertensão de difícil controle ou com complic...,1.0,Comorbidades,213VCD027Z,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-05-26,1ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-06-04T16:31:21.000Z,16341
3,c2f62bfc-ea2f-49a4-8b5e-231fd81d6525-i0b0,0960686bb61eed85e4ffcda5a8bae03ab44f59ec37759e...,19.0,2002-07-17,F,3.0,PARDA,150330.0,10.0,IGARAPE-MIRI,BRASIL,PA,68430.0,B,2767007,PREFEITURA MUNICIPAL DE IGARPEMIRI,UNIDADE DE VIGILANCIA EM SAUDE DE IGARAPEMIRI,150330,IGARAPE-MIRI,PA,201,Pessoas de 60 a 64 anos,2.0,Faixa Etária,215VCD144W,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-08-16,2ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-08-19T20:00:08.000Z,16341
4,25f4db29-a63f-4abc-967e-4fafe51a2068-i0b0,0c2525845d2bda7faad09d6a36228e474ed7314e51c12d...,32.0,1988-11-04,M,2.0,PRETA,150140.0,10.0,BELEM,BRASIL,PA,66110.0,B,7780966,DEPARTAMENTO DE VIGILANCIA A SAUDE,DEVS,150140,BELEM,PA,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,FA9094,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-10,1ª Dose,87,Vacina covid-19 - BNT162b2 - BioNTech/Fosun Ph...,Novo PNI,2021-07-13T21:56:23.000Z,16341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4699070,30ee1860-020f-48a7-b3f0-35f5a8d36ac7-i0b0,99a24a39ed78cac06747297ffee2149641482acb620782...,72.0,1949-02-01,M,99.0,SEM INFORMACAO,150178.0,10.0,BREU BRANCO,BRASIL,PA,68488.0,B,2678500,MUNICIPIO DE BREU BRANCO,UNIDADE MISTA DE SAUDE DR INACIO GABRIEL,150178,BREU BRANCO,PA,203,Pessoas de 70 a 74 anos,2.0,Faixa Etária,212VCD002ZVB,FUNDACAO OSWALDO CRUZ,Organization/33781055000135,2021-03-26,1ª Dose,85,Vacina Covid-19 - Covishield,Novo PNI,2021-03-31T16:25:45.000Z,16341
4699071,3a3c3c54-449f-437e-a74b-059a34e0b2d4-i0b0,38cb19219ab629b563b44b04a4fe260624e56e567af34f...,30.0,1990-10-11,M,99.0,SEM INFORMACAO,310620.0,10.0,BELO HORIZONTE,BRASIL,MG,31035.0,B,106585,MUNICIPIO DE BARCARENA,CENTRAL DE REDE DE FRIOS MUNICIPAL,150130,BARCARENA,PA,1601,Trabalhadores In

In [ ]:
## Generating age groups
df_si_pni['faixa_etaria'] = pd.cut(x = df_si_pni.paciente_idade, 
                                      bins = [-1, 9, 19, 29, 39, 49, 59, 69, 79, 200], 
                                      labels = ['<9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', "90+"])

## Merging vaccine names

df_si_pni = df_si_pni.assign(
            vacina_grupo = np.select(
                [df_si_pni['vacina_codigo'] == 85, 
                 df_si_pni['vacina_codigo'] == 86, 
                 df_si_pni['vacina_codigo'] == 87,
                 df_si_pni['vacina_codigo'] == 89, 
                 df_si_pni['vacina_codigo'] == 88],
                ['AZ', 'Sinovac/Coronavac', 'Pfizer', 'AZ', 'Janssen'],
                default="Other"
                ),
            vacina_dose1 = np.select(
                [df_si_pni['vacina_descricao_dose'] == '1ª\xa0Dose'], 
                ["Yes"],
                default=""
                ),
            vacina_dose2 = np.select(
                [df_si_pni['vacina_descricao_dose'] == '2ª\xa0Dose'], 
                ["Yes"],
                default=""
                ),
            vacina_dose_unica = np.select(
                [df_si_pni['vacina_descricao_dose'] == 'Dose\xa0'], 
                ["Yes"],
                default=""
                )
            )

print(df_si_pni.shape)

In [ ]:

#### Treating duplicated rows inputted from different systems
df_novo_pni = df_si_pni.loc[(df_si_pni['sistema_origem'].isin(['Novo PNI']))]
df_antigo = df_si_pni.loc[(df_si_pni['sistema_origem'].isin(['ESUS APS - NACIONAL (OFFLINE)'])) ]

df_antigo_fora_do_novo = df_antigo.loc[~df_antigo['paciente_id'].isin(df_novo_pni['paciente_id'].to_list()) ]

df_si_pni = pd.concat( [df_novo_pni,  df_antigo_fora_do_novo] )

# Size values
print(df_novo_pni.shape)
print(df_antigo.shape)
print(df_antig_fora_do_novo.shape)
print(df_si_pni.shape)




(4699075, 39)
(4686597, 39)
(12478, 39)
(2834, 39)
(4689431, 39)


In [ ]:
## Treating duplicate id_values
df_id_counts = df_si_pni['paciente_id'].value_counts().reset_index()
df_id_counts_3 = df_id_counts.loc[df_id_counts['paciente_id'] == 3]['index'].to_list()

#print(df_id_counts_3)

#df_si_pni.loc[df_si_pni['paciente_id'] == df_id_counts_3[3]]

df_si_pni.loc[(df_si_pni['paciente_datanascimento'] == '1995-08-14') & (df_si_pni['paciente_endereco_nmmunicipio'] == 'BELEM') & 
              (df_si_pni['paciente_enumsexobiologico'] == 'M')]

#df_si_pni.loc[ df_si_pni['paciente_id'].isin(['409a5ce781911a41abfad77c601542e833eda0304f1bd5c3a172ab1776371468'])]


,document_id,paciente_id,paciente_idade,paciente_datanascimento,paciente_enumsexobiologico,paciente_racacor_codigo,paciente_racacor_valor,paciente_endereco_coibgemunicipio,paciente_endereco_copais,paciente_endereco_nmmunicipio,paciente_endereco_nmpais,paciente_endereco_uf,paciente_endereco_cep,paciente_nacionalidade_enumnacionalidade,estabelecimento_valor,estabelecimento_razaosocial,estalecimento_nofantasia,estabelecimento_municipio_codigo,estabelecimento_municipio_nome,estabelecimento_uf,vacina_grupoatendimento_codigo,vacina_grupoatendimento_nome,vacina_categoria_codigo,vacina_categoria_nome,vacina_lote,vacina_fabricante_nome,vacina_fabricante_referencia,vacina_dataaplicacao,vacina_descricao_dose,vacina_codigo,vacina_nome,sistema_origem,data_importacao_rnds,id_sistema_origem,faixa_etaria,vacina_grupo,vacina_dose1,vacina_dose2,vacina_dose_unica
36976,ef64f3eb-3b7e-4fba-9ab1-812bc153ad3f-i0b0,b3178702dd762c9246c1361a0593ec30ebdf0ece796c0d...,25.0,1995-08-14,M,1.0,BRANCA,150140.0,10.0,BELEM,BRASIL,PA,66825.0,B,7780966,DEPARTAMENTO DE VIGILANCIA A SAUDE,DEVS,150140,BELEM,PA,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,210197,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-30,1ª Dose,89,Covid-19-AstraZeneca,Novo PNI,2021-08-01T23:57:49.000Z,16341,20-29,AZ,Yes,,
179008,138ba4a4-a5dd-4df0-9fcf-dbb490367dd9-i0b0,5fef81362cf21eecae7ccb14231e73678936f9c08e4cd4...,25.0,1995-08-14,M,4.0,AMARELA,150140.0,10.0,BELEM,BRASIL,PA,66033.0,B,7780966,DEPARTAMENTO DE VIGILANCIA A SAUDE,DEVS,150140,BELEM,PA,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,210197,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-30,1ª Dose,89,Covid-19-AstraZeneca,Novo PNI,2021-08-02T03:45:49.000Z,16341,20-29,AZ,Yes,,
503886,be5fbea5-f6e3-4ce7-b259-da19912a4db0-i0b0,5436bb3878a66d61545f7ac2b18770a579dc381d0d5a31...,25.0,1995-08-14,M,99.0,SEM INFORMACAO,150140.0,10.0,BELEM,BRASIL,PA,66045.0,B,7780966,DEPARTAMENTO DE VIGILANCIA A SAUDE,DEVS,150140,BELEM,PA,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,210197,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-30,1ª Dose,89,Covid-19-AstraZeneca,Novo PNI,2021-08-01T21:24:12.000Z,16341,20-29,AZ,Yes,,
548849,09d1278b-0f0d-459e-8d7d-d174f164e373-i0b0,46f05f3b7f5e5859b91203d7af7aabe081a8ee8921af0b...,25.0,1995-08-14,M,3.0,PARDA,150140.0,10.0,BELEM,BRASIL,PA,66821.0,B,7780966,DEPARTAMENTO DE VIGILANCIA A SAUDE,DEVS,150140,BELEM,PA,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,210197,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-30,1ª Dose,89,Covid-19-AstraZeneca,Novo PNI,2021-08-02T01:04:42.000Z,16341,20-29,AZ,Yes,,
777617,071fc10e-ddc4-4d51-a768-08194e890c75-i0b0,8807e29b3a7aa82f393aea62dbda301c2fac595cc1fd37...,25.0,1995-08-14,M,99.0,SEM INFORMACAO,150140.0,10.0,BELEM,BRASIL,PA,66070.0,B,7780966,DEPARTAMENTO DE VIGILANCIA A SAUDE,DEVS,150140,BELEM,PA,801,Ensino Básico,8.0,Trabalhadores da Educação,EW0199,MINISTERIO DA SAUDE,Organization/00394544000851,2021-05-29,1ª Dose,87,Vacina covid-19 - BNT162b2 - BioNTech/Fosun Ph...,Novo PNI,2021-06-01T05:32:20.000Z,16341,20-29,Pfizer,Yes,,
1101260,a4c8ce5c-3985-4aae-8ae0-1ffef6d17564-i0b0,25aed372f5c3f17a4a0609a2983b2f38a59c4c0501d623...,25.0,1995-08-14,M,99.0,SEM INFORMACAO,150140.0,10.0,BELEM,BRASIL,PA,66110.0,B,7780966,DEPARTAMENTO DE VIGILANCIA A SAUDE,DEVS,150140,BELEM,PA,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,210197,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-30,1ª Dose,89,Covid-19-AstraZeneca,Novo PNI,2021-08-01T17:05:36.000Z,16341,20-29,AZ,Yes,,
1302403,c0c849db-b1c8-4e58-b682-85b2b02deebc-i0b0,caf65b34fa25c88dcda49de583c04b650dedb3aae42b61...,25.0,1995-08-14,M,3.0,PARDA,150140.0,10.0,BELEM,BRASIL,PA,66083.0,B,7780966,DEPARTAMENTO DE VIGILANCIA A SAUDE,DEVS,150140,BELEM,PA,201,Pessoas de 18 a 64 anos,2.0,Faixa Etária,210197,MINISTERIO DA SAUDE,Organization/00394544000851,2021-07-30,1ª Dose,89,Covid-19-AstraZeneca,Novo PNI,2021-08-01T18:53:59.000Z,16341,20-29,AZ,Yes,,
1538718,151454eb-36b7-4a3f-a9f8-8403eb05f46a-i0b0,9535e8c04ea8a5df3f32